In [21]:
import findspark                                              #Import library to Search for Spark Installation  

findspark.init()                                              #Search Spark Installation

import pyspark                                                #Only run after findspark.init()

from pyspark.sql import SparkSession                          # Import of Spark Session
from pyspark import SparkContext as spark                     # Import the Regular Spark Contex 
from pyspark.sql import SQLContext                            # Import the SQL Spark Contex 
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.sql.functions as F
spark = SparkSession.builder.getOrCreate()
import os

import pyarrow.parquet as pq                                  #Create Parquet File

sc = spark.sparkContext                                       #Initialize Spark 

spark.conf.set("spark.sql.broadcastTimeout",  36000)
#conf = pyspark.SparkConf().setMaster("yarn-client").setAppName("sparK-mer")
#conf.set("spark.executor.heartbeatInterval","3600s")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-------------

# Creating Variables

## Buckets

In [ ]:
SPG_INTEGRATION_INPUT_BUCKET = os.environ['INTEGRATION_INPUT_BUCKET']

In [ ]:
SPG_OUTPUT_BUCKET = os.environ['OUTPUT_BUCKET']

## Input Paths

In [ ]:
# SPG Regions
SPG_INTEGRATION_INPUT_BUCKET_REGION = "SPG_DIMENSIONS/SPG_REGIOES/SPG_RG_REGIOES/SPG_RG_REGIOES.parquet"

In [ ]:
# SPG Hana
SPG_INTEGRATION_INPUT_BUCKET_HANA = "SPG_FACTS/SPG_HANA/SPG_HANA.parquet"

## Output Paths

In [ ]:
SPG_OUTPUT_BUCKET_ALERT = "SPG_ALERT_DESAD/tb_SPG_AD_DESADERENCIA.parquet"

------------

# Importing Tables

In [22]:
# Reading parquet file from the SandBox Data Lake
df_hana = spark.read.parquet("s3a://"+SPG_INTEGRATION_INPUT_BUCKET+"/"+SPG_INTEGRATION_INPUT_BUCKET_HANA)   #Billing Hana 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# SPG Regions
df_region = spark.read.parquet("s3a://"+SPG_INTEGRATION_INPUT_BUCKET+"/"+SPG_INTEGRATION_INPUT_BUCKET_REGION)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------

# Preparing Table

In [24]:
# Filter the Sales Organizations within Smart Pricing scope
df_hana=df_hana.filter(df_hana.SalesOrg.like('%BRIN%') |
                       df_hana.SalesOrg.like('%BRIO%') |
                       df_hana.SalesOrg.like('%BRDI%') |
                       df_hana.SalesOrg.like('%BRDO%') |
                       df_hana.SalesOrg.like('%BRCC%') |
                       df_hana.SalesOrg.like('%BRCO%') |
                       df_hana.SalesOrg.like('%BRCG%') |
                       df_hana.SalesOrg.like('%BRGO%'))\
                .filter(df_hana.BillingType.like('%ZF2B%'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df_hana = df_hana.withColumn(col_name, col(col_name).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
df_hana=df_hana.filter(df_hana.MA_BillingQuantity_P > 0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
#Rename columns imported from hana
df_hana=df_hana.withColumnRenamed("SalesOrg", "SALES_ORG_COD")\
               .withColumnRenamed("MA_CM_PRECOLISTA_D", "LIST_PRICE")\
               .withColumnRenamed("DP_CM_PRECOPRATICADO_D", "PRACTICED_PRICE")\
               .withColumnRenamed("IM_CM_CD_ICMS_D", "ICMS")\
               .withColumnRenamed("DA_CM_CD_ZD01_D", "ZD01")\
               .withColumnRenamed("DA_CM_CD_ZD02_D", "ZD02")\
               .withColumnRenamed("DA_CM_CD_ZD03_D", "ZD03")\
               .withColumnRenamed("DA_CM_CD_ZD04_D", "ZD04")\
               .withColumnRenamed("DA_CM_CD_ZD05_D", "ZD05")\
               .withColumnRenamed("DA_CM_CD_ZD06_D", "ZD06")\
               .withColumnRenamed("AA_CM_CD_ZEB1_D", "ZEB1")\
               .withColumnRenamed("AA_CM_CD_ZEB2_D", "ZEB2")\
               .withColumnRenamed("AA_CM_CD_ZEAF_D", "ZEAF")\
               .withColumnRenamed("DM_CM_CD_ZD13_D", "ZD13")\
               .withColumnRenamed("DM_CM_CD_ZD14_D", "ZD14")\
               .withColumnRenamed("DM_CM_CD_ZP30_D", "ZP30")\
               .withColumnRenamed("AM_CM_CD_ZSU2_D", "ZSU2")\
               .withColumnRenamed("MA_CM_CD_ZP20_D", "ZP20")\
               .withColumnRenamed("MA_CM_CD_BX41_D", "BX41")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# Correcting "Ouro Branco" Sales Org 
    # BRIO -> BRIN
    # BRCO -> BRCC
    # BRDO -> BRDI
    # BRGO -> BRCG
df_hana=df_hana.withColumn("SALES_ORG_COD", when(df_hana.SALES_ORG_COD.like("BRCO"), "BRCC")\
                                              .otherwise(when(df_hana.SALES_ORG_COD.like("BRGO"), "BRCG")\
                                                        .otherwise(when(df_hana.SALES_ORG_COD.like("BRIO"), "BRIN")\
                                                                  .otherwise(when(df_hana.SALES_ORG_COD.like("BRDO"), "BRDI")
                                                                            .otherwise(df_hana.SALES_ORG_COD)))))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# Filter the GPD within Smart Pricing scope
df_hana = df_hana.filter(df_hana.GPDName.like('%BR-PERFIS ESTRUTURAIS%') |
df_hana.GPDName.like('%BR-CA50%') |
df_hana.GPDName.like('%BR-CA60%') |
df_hana.GPDName.like('%BR-RECOZIDO%') |
df_hana.GPDName.like('%BR-TEL TUBO%') |
df_hana.GPDName.like('%BR-TELA P/ CONCRETO%') |
df_hana.GPDName.like('%BR-CORTE E DOBR%') |
df_hana.GPDName.like('%BR-CONSTRUCAO CIVIL%') |
df_hana.GPDName.like('%BR-CA25%') |
df_hana.GPDName.like('%BR-PERFIS COMERCIAIS%') |
df_hana.GPDName.like('%BR-BTG%') |
df_hana.GPDName.like('%BR-MALHA POP%') |
df_hana.GPDName.like('%BR-TELA COLUNA%') |
df_hana.GPDName.like('%BR-TRELIÇA%') |
df_hana.GPDName.like('%BR-B.TREFILADA%') |
df_hana.GPDName.like('%BR-OVALADO%') |
df_hana.GPDName.like('%BR-GALVANIZADO%') |
df_hana.GPDName.like('%BR-BARRAS COMERCIAIS%') |
df_hana.GPDName.like('%BR-CAIXOTARIA%') |
df_hana.GPDName.like('%BR-ARTEFATOS%') |
df_hana.GPDName.like('%BR-FARPADO%') |
df_hana.GPDName.like('%BR-ELETRODO%') |
df_hana.GPDName.like('%BR-SOLDA - MIG%') |
df_hana.GPDName.like('%BR-CANTON A572%') |
df_hana.GPDName.like('%BR-GGS%') |
df_hana.GPDName.like('%BR-ARMADO%') |
df_hana.GPDName.like('%BR-CORDOALHA%') |
df_hana.GPDName.like('%BR-ESTACA PRANCHA%') |
df_hana.GPDName.like('%BR-ARAME PREGO%') |
df_hana.GPDName.like('%BR-CABEÇA DUPLA%') |
df_hana.GPDName.like('%BR-CORDOALHA AGRO%') |
df_hana.GPDName.like('%BR-GRAMPO%') |
df_hana.GPDName.like('%BR-COBREADOS%') |
df_hana.GPDName.like('%BR-CHAPA LQ%') |
df_hana.GPDName.like('%BR-UDC%') |
df_hana.GPDName.like('%BR-CHAPA ZN%') |
df_hana.GPDName.like('%BR-TELHA AZ%') |
df_hana.GPDName.like('%BR-TUBO ZN%') |
df_hana.GPDName.like('%BR-MARCENARIA%') |
df_hana.GPDName.like('%BR-PREGOES%') |
df_hana.GPDName.like('%BR-TELHEIRO%') |
df_hana.GPDName.like('%BR-COLUNA%') |
df_hana.GPDName.like('%BR-ESTRIBO%') |
df_hana.GPDName.like('%BR-ACESSORIOS%') |
df_hana.GPDName.like('%BR-CHAPA LCG%') |
df_hana.GPDName.like('%BR-CHAPA LF%') |
df_hana.GPDName.like('%BR-TUBO LF%') |
df_hana.GPDName.like('%BR-CHAPA LQ PISO%') |
df_hana.GPDName.like('%BR-BOBININHA%') |
df_hana.GPDName.like('%BR-ESPECIAIS%') |
df_hana.GPDName.like('%BR-BOBINA LQ%') |
df_hana.GPDName.like('%BR-FITA LQ%') |
df_hana.GPDName.like('%BR-BOBINA AZ%') |
df_hana.GPDName.like('%BR-AÇOS ESPECIAIS%') |
df_hana.GPDName.like('%BR-PARAFUSOS%') |
df_hana.GPDName.like('%BR-CIMENTO%') |
df_hana.GPDName.like('%BR-TUBO LQ%') |
df_hana.GPDName.like('%BR-TELHA ZN%') |
df_hana.GPDName.like('%BR-BTC GLV CP%') |
df_hana.GPDName.like('%BR-BOBINA ZN%') |
df_hana.GPDName.like('%BR-S-BARRA LAMINADA MÉDIA%') |
df_hana.GPDName.like('%BR-S-BARRA LAMINADA PESADA%') |
df_hana.GPDName.like('%BR-FITA LF%') |
df_hana.GPDName.like('%BR-FITA AZ%') |
df_hana.GPDName.like('%BR-S-BARRA LAMINADA LEVE%') |
df_hana.GPDName.like('%BR-ALAMBRADO%') |
df_hana.GPDName.like('%BR-SAPATA%') |
df_hana.GPDName.like('%BR-MOURÃO%') |
df_hana.GPDName.like('%BR-ATC CLARO IND%') |
df_hana.GPDName.like('%BR-POLIDO%') |
df_hana.GPDName.like('%BR-PERFIL BENEFICIADO%') |
df_hana.GPDName.like('%BR-BOBINA LF%') |
df_hana.GPDName.like('%BR-S-BARRA CTT%') |
df_hana.GPDName.like('%BR-CHAPA AZ%') |
df_hana.GPDName.like('%BR-FITA ZN%') |
df_hana.GPDName.like('%BR-LDA%') |
df_hana.GPDName.like('%BR-PIATINA CLARA%') |
df_hana.GPDName.like('%BR-S-BARRA DESCASCADA%') |
df_hana.GPDName.like('%BR-MESH%') |
df_hana.GPDName.like('%BR-S-BARRA FORJADA FINA%') |
df_hana.GPDName.like('%BR-S-BARRA RETIFICADA%') |
df_hana.GPDName.like('%BR-S-BARRA TREFILADA%') |
df_hana.GPDName.like('%BR-PLACA%') |
df_hana.GPDName.like('%BR-COIL%') |
df_hana.GPDName.like('%BR-S-FERRAMENTA%') |
df_hana.GPDName.like('%BR-S-FIO MÁQUINA STT%') |
df_hana.GPDName.like('%BR-ATC CLARO MOL COL%') |
df_hana.GPDName.like('%BR-ATC GLV CP%') |
df_hana.GPDName.like('%BR-B.CHATA LONGARINA%') |
df_hana.GPDName.like('%BR-BT FORJARIA BLOCOS%') |
df_hana.GPDName.like('%BR-TARUGO%') |
df_hana.GPDName.like('%BR-DRAWING%') |
df_hana.GPDName.like('%BR-ATC CLARO ENF ALG%') |
df_hana.GPDName.like('%BR-PIATINA GLV%') |
df_hana.GPDName.like('%BR-PERFIL DORMENTE%') |
df_hana.GPDName.like('%BR-ATC%') |
df_hana.GPDName.like('%BR-CHQ%') |
df_hana.GPDName.like('%BR-ALMA DE ELETRODO%') |
df_hana.GPDName.like('%BR-FIO MAQUINA%') |
df_hana.GPDName.like('%BR-CHQ BORO%') |
df_hana.GPDName.like('%BR-PERFIL GUIA ELEV%'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# Filter the GPM within Smart Pricing scope
df_hana = df_hana.filter(df_hana.GPM.like('%PERFIS_ESTRUTURAIS%') |
df_hana.GPM.like('%VERGALHAO%') |
df_hana.GPM.like('%AMPLIADOS%') |
df_hana.GPM.like('%CORTE_DOBRA%') |
df_hana.GPM.like('%PREGOS%') |
df_hana.GPM.like('%B&P%') |
df_hana.GPM.like('%ARAMES_AGRO%') |
df_hana.GPM.like('%ARAMES_IND%') |
df_hana.GPM.like('%PLANOS_LQ%') |
df_hana.GPM.like('%PLANOS_REVESTIDOS%') |
df_hana.GPM.like('%PLANOS_LF%') |
df_hana.GPM.like('%PLANOS_LCG%') |
df_hana.GPM.like('%FIO_MAQUINA%') |
df_hana.GPM.like('%BT_FORJARIA%') |
df_hana.GPM.like('%PLACAS%') |
df_hana.GPM.like('%TARUGO%'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# Creating columns Strategic Price and Practiced Price
df_hana=df_hana.withColumn("STRATEGIC_PRICE", when((df_hana.SALES_ORG_COD.like("BRCG")), col("LIST_PRICE")+col("ZD02")+col("ZD03")+col("ZEB2")+col("BX41"))\
                                        .otherwise(col("LIST_PRICE")+col("ZD02")+col("ZD03")+col("ZEB2")))\
               .withColumn("PRACTICED_PRICE", when((df_hana.SALES_ORG_COD.like("BRCG")), 
                                                   col("PRACTICED_PRICE")+abs(col("ZD05"))-col("ZEAF")+col("BX41"))\
                                              .otherwise(col("PRACTICED_PRICE")+abs(col("ZD05"))-col("ZEAF")))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# Selecting necessary columns from Hana Table
df_hana=df_hana.select(df_hana.SALES_ORG_COD
                       ,df_hana.SoldToCity.alias("ISSUING_CITY")
                       ,df_hana.SoldToState.alias("ISSUING_STATE")
                       ,df_hana.SalesOrderDate.alias("SALES_ORDER_DATE")
                       ,df_hana.ShipToCustomerName.alias("CUSTOMER")
                       ,df_hana.Material.alias("MATERIAL_COD")
                       ,df_hana.MaterialName.alias("MATERIAL_DESC")
                       ,df_hana.MA_BillingQuantity_P.alias("QUANTITY_TON")
                       ,df_hana.BILDOCDATE.alias("BILLING_DATE")
                       ,df_hana.BilDocNumber.alias("BilDocNumber")
                       ,df_hana.BilDocItem.alias("BilDocItem")
                       ,df_hana.MA_CM_RBV_D.alias("RBV")
                       ,df_hana.STRATEGIC_PRICE
                       ,df_hana.PRACTICED_PRICE
                       ,df_hana.LIST_PRICE
                       ,df_hana.GPM
                       ,df_hana.GPDName.alias("GPD_DESC")
                       ,df_hana.SalesDocNumber.alias("SALES_DOC_NUMBER")
                       ,df_hana.MA_CM_PRECOBASE_ALCADA_P.alias("VERGE")
                       ,df_hana.MA_CM_DESVIOPOLITICAALCADA_D.alias("VERGE_POLICY_DEVIATION")
                       ,df_hana.ICMS
                       ,df_hana.ZD01
                       ,df_hana.ZD02
                       ,df_hana.ZD03
                       ,df_hana.ZD04
                       ,df_hana.ZD05
                       ,df_hana.ZD06
                       ,df_hana.ZEB1
                       ,df_hana.ZEB2
                       ,df_hana.ZEAF
                       ,df_hana.ZD13
                       ,df_hana.ZD14
                       ,df_hana.ZP30
                       ,df_hana.ZSU2
                       ,df_hana.ZP20
                       ,df_hana.BX41)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df_region=df_region.select(df_region["IBGE_UF_ACRONYMS"].alias("STATE_drop")
                           ,df_region["BRANCH"].alias("BRANCH")
                           ,df_region["IBGE_SAP_CITY_NAME"].alias("CITY_DROP")).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df_region=df_region.withColumn("REGION", expr("substring(BRANCH, 7, length(BRANCH))"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df_hana = df_hana.join(df_region, (df_hana["ISSUING_CITY"] == df_region["CITY_DROP"]) &
                                  (df_hana["ISSUING_STATE"] == df_region["STATE_drop"]), how='inner')\
                 .drop("STATE_drop")\
                 .drop("CITY_DROP")\
                 .drop("ISSUING_CITY")\
                 .drop("ISSUING_STATE")\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df_hana=df_hana.distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
df_hana.write.mode('overwrite').parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_ALERT)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…